In [43]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import json
import requests

In [44]:
req_list = [
    'Total population',
    'Median age (years)',
    'Average  household size',
 'Average family size',
    'Population 25 years and over',
 'Less than high school graduate',
 'High school graduate (includes equivalency)',
 "Some college or associate's degree",
 "Bachelor's degree",
 'Graduate or professional degree',
     'Median Household income (dollars)',
 'Owner-occupied housing units',
 'Renter-occupied housing units',
]

In [59]:
len(req_list)

13

In [45]:
def clean(x):
    try:
        x = str(x)
        x = x.replace(',','').replace('%','')
        x = float(x)
        return x
    except:
        return x
def extract_data(file):
    df  = pd.read_csv(file)
    df = df[[i for i in df.columns if  'Total!!Estimate' in i or 'Label' in i]]
    df = df.T
    df.columns = df[:1].values[0]
    df  = df[1:]
    df.columns = [i.replace('\xa0','') for i in df.columns]
    burden = df.iloc[:,-7:]
    burden.columns = ['SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS',
                      'less_than_30%_Owner','over_30%_Owner','Renter-occupied housing units',
                      'GROSS RENT AS A PERCENTAGE OF HOUSEHOLD INCOME IN THE PAST 12 MONTHS',
                      'less_than_30%_renter',
                      'over_30%_renter']
    burden = burden[[ 'less_than_30%_Owner','over_30%_Owner','less_than_30%_renter',
                      'over_30%_renter']]
    final = df[req_list]
    final = pd.concat([final,burden],axis=1)
    for cols in final.columns:
        final[cols] = final[cols].map(lambda x: clean(x))
    return final
    
    

In [46]:
files = os.listdir('./housing-data')
files = ['./housing-data/'+ file for file in files if '.csv' in file]

In [47]:
files

['./housing-data/ACSST1Y2019.S0501-2024-11-10T171737.csv',
 './housing-data/ACSST1Y2022.S0501-2024-11-10T173552.csv',
 './housing-data/ACSST5Y2020.S0501-2024-11-10T171707.csv',
 './housing-data/ACSST1Y2021.S0501-2024-11-10T171641.csv',
 './housing-data/ACSST1Y2023.S0501-2024-11-10T171602.csv']

In [48]:
from tqdm import tqdm
data = pd.DataFrame()
for file in tqdm(files):
    d = extract_data(file)
    year = file.split('.')[1][-4:]
    d['year'] = year
    data = pd.concat([data,d],axis=0)
    
    

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 55.71it/s]


In [49]:
data = data.reset_index()
data = data.rename(columns={'index':'Metro'})

In [50]:

data['Metro'] = data.Metro.map(lambda metro: metro.split(',')[0])


In [51]:
metro_mapper = {
    'Miami-Fort Lauderdale-West Palm Beach':'Miami-Fort Lauderdale-Pompano Beach',
    'Houston-Pasadena-The Woodlands':'Houston-The Woodlands-Sugar Land',
    'Denver-Aurora-Centennial':'Denver-Aurora-Lakewood',
    'Austin-Round Rock-San Marcos':'Austin-Round Rock-Georgetown',
    'Atlanta-Sandy Springs-Roswell':'Atlanta-Sandy Springs-Alpharetta'}


In [52]:
data['Metro'] = data.Metro.map(lambda x: metro_mapper[x] if metro_mapper.get(x) is not None else x)

In [53]:
data.to_csv('HousingData.csv',index=False)

In [70]:
df = pd.read_csv('HousingData.csv')
df = df.drop(['Owner-occupied housing units.1','Renter-occupied housing units.1'],axis=1)



In [73]:
metros = pd.DataFrame(df.Metro.unique()).reset_index()
metros.columns = ['geoID','Metro']
metros['geoID'] += 1

In [74]:
df = pd.merge(metros,df)

In [76]:
df.to_csv('HousingData.csv',index=False)
metros.to_csv('Metros.csv',index=False)